In [2]:
pip install langchain-google-genai google-generativeai langchain-core


  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached langchain_google_genai-2.1.10-py3-none-any.whl.metadata (7.2 kB)
  Attempting uninstall: langchain-google-genai
    Found existing installation: langchain-google-genai 2.1.10
    Uninstalling langchain-google-genai-2.1.10:
      Successfully uninstalled langchain-google-genai-2.1.10
Note: you may need to restart the kernel to use updated

In [3]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

In [9]:
from transformers import pipeline
llm=pipeline('text-generation', model="gpt2")
result = llm("What is exact location of Titanic Ship?", max_new_tokens=50)
print(result[0]['generated_text'])

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is exact location of Titanic Ship?

The Titanic ship in the picture is in the port of London, which was built by the British government in 1898, and has been the site of some of the world's greatest historic works. It is a large ship, about 20 stories high


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

inputs = tokenizer("What is the exact location of the indian ocean?", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the exact location of the indian ocean?

The indian ocean is a large, shallow, and shallow ocean. It is the largest ocean in the world. It is the largest ocean in the world. It is the largest ocean in the world. It is the largest ocean in the


In [29]:
pip install -U langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [67]:
# from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.document_loaders import PyPDFLoader

In [68]:
loader=PyPDFLoader(r"C:\Users\User\Downloads\Updated-Langchain-main\Updated-Langchain-main\rag\attention.pdf")
docs=loader.load()

In [69]:
len(docs)

15

In [70]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100
)
chunks =splitter.split_documents(docs)

In [71]:
len(chunks)

378

In [ ]:
splitter=RecursiveCharacterTextSplitter(chunk_size=200,
chunk_overlap=100)
split = splitter.split_documents(document)

In [72]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
model.save("models/all-MiniLM-L6-v2")
#"sentence-transformers/all-MiniLM-L6-v2"

In [73]:
embeddings= HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [77]:
vectorstore=Chroma.from_documents(chunks, embedding=embeddings)

In [75]:
a1=vectorstore.get()

In [76]:
print(a1)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [78]:
print(vectorstore)

In [79]:
model=pipeline('text-generation', model='gpt2', max_new_tokens=256)
llm=HuggingFacePipeline(pipeline=model)

Device set to use cpu


In [ ]:
print(llm.pipeline)

In [81]:
print(llm.pipeline.model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [83]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff",
)

In [84]:
while True:
    query = input("Ask a question (or 'exit'): ")
    if query.lower() == "exit":
        break
    answer = qa.run(query)
    print("\n🤖 Answer:", answer, "\n")

Ask a question (or 'exit'):  attention mechnism


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗

scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗

Menswear

Question: attention mechnism
Helpful Answer: Attention is all you need to be in the right place. But you need to be mindful.

Ashish Vaswani∗

Google Brain

avaswani@google.com

Noam Shazeer∗

Google Brain

noam@google.com

Niki Parmar∗

Google Research

nikip@google.com

Jakob Uszkoreit∗

Scholarly Works

Question: mindfulness is the mind

Helpful Answer: You need to be mindful. If you are mindful, you can avoid 

Ask a question (or 'exit'):  exit


In [ ]:
#for excel file

In [1]:
import pandas as pd
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

In [2]:
df=pd.read_excel(r"C:\Users\User\Downloads\Retail Line Print Week 2614.xlsx",skiprows=1)

In [3]:
df.shape

(80016, 75)

In [4]:
df.columns

Index(['Item No + Colour', 'Season', 'Cat. Code', 'Category', 'Sub Group Code',
       'Sub Group', 'Item No', 'Style Name', 'Colour', 'Vendor Name', 'Story',
       'Selling Month', 'Newness', 'Replenishment Grade Code', 'No Of\nStores',
       'Item\nLifecycle\nStatus', 'Landed Cost', 'Current Price',
       'Original Price', 'Wholesale Price', 'Margin', 'Fp Md', 'Sales (£K)',
       'Sales\nVar LW', 'Units', 'Wk -1 Units', 'Wk -2 Units', 'ROS',
       'Planned ROS', 'Shop Stock', 'Shop Cover', 'Intransit Stock',
       'At Warehouse', 'Clearance Stock', 'All Channel Warehouse Stock',
       'Total Cover', 'Weeks On Sale', 'Total Sell Through', 'Cum Sales Units',
       'Cum Sales\nValue (£K)', 'PO Unit\nReceived LW', 'PO £\nReceived LW',
       'Retail PO\nReceived (Cum)', 'Retail Value\nReceived (Cum)',
       'Commitment', 'DC Date', 'OTB', 'Wholesale\nPO Received',
       'Wholesale \nValue Received', 'Wholesale\nWarehouse\n Stock',
       'Home Sales (U)', 'Home Demand (U)', 'Ho

In [40]:
docs=[Document(page_content=row) for row in df['L1 Descr'].astype(str)]

In [41]:
splitter=RecursiveCharacterTextSplitter(chunk_size=200,
                                       chunk_overlap=100)
split=splitter.split_documents(docs)

In [42]:
docs

[Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_content='Womenswear'),
 Document(metadata={}, page_con

In [43]:
len(split)

80016

In [44]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [45]:
vectorstore = Chroma.from_documents(split, embedding=embeddings)
#split

In [49]:
vectorstore = Chroma.from_documents(docs, embedding=embeddings)
#docs

In [50]:
len(vectorstore)

240048

In [51]:
collection = vectorstore._collection
all_docs = collection.get(include=["metadatas", "documents"])

for i, text in enumerate(all_docs["documents"][:3]): 
    print(f"\n📄 Document {i+1}:")
    print(text)


📄 Document 1:
433387ORANGE MLT

📄 Document 2:
439439BLUE MLT

📄 Document 3:
439562BLUE MLT


In [52]:
query = "What is L1 Descr?"
results = vectorstore.similarity_search(query, k=3)
print(results[0])
for i, doc in enumerate(results):
    print(f"\n🔹 Match {i+1}:")
    print(doc.page_content)

page_content='442121TEAL MLT'

🔹 Match 1:
442121TEAL MLT

🔹 Match 2:
407341LT LAVENDE

🔹 Match 3:
442102BLK MLT


In [53]:
len(vectorstore)

240048

In [54]:
model=pipeline("text-generation", model="gpt2", max_new_tokens=200)

Device set to use cpu


In [56]:
llm = HuggingFacePipeline(pipeline = model)
# llm("Hello world", max_new_tokens=50, temperature=0.7)


In [57]:
qa= RetrievalQA.from_chain_type(
    llm=llm,
    retriever = vectorstore.as_retriever(search_kwargs={"k":3})
)

In [58]:
while True:
    query = input("Ask a question (or 'exit'): ")
    if query.lower() == "exit":
        break
    answer = qa.run(query)
    print("\n🤖 Answer:", answer, "\n")

Ask a question (or 'exit'):  how many category are there


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Unknown

Unknown

Unknown

Question: how many category are there
Helpful Answer: 1.0

2.0

3.0

4.0

5.0

6.0

7.0

8.0

9.0

10.0

11.0

12.0

13.0

14.0

15.0

16.0

17.0

18.0

19.0

20.0

21.0

22.0

23.0

24.0

25.0

26.0

27.0

28.0

29.0

30.0

31.0

32.0

33.0

34.0

35.0

36.0

37.0

38.0

39.0

40.0

 



Ask a question (or 'exit'):  what is the sum of total catgory


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

427163PLUM

203782COCOA BEAN

284290CHARCOAL

Question: what is the sum of total catgory
Helpful Answer:

428092COCOA BEAN

284290CHARCOAL

362218COCOA BEAN

210650V

4

CATGORY

4

CATGORY

362218COCOA BEAN

210650V

4

CATGORY

36411COCOA BEAN

284290CHARCOAL

4

CATGORY

36411COCOA BEAN

210650V

4

CATGORY

36411COCOA BEAN

210650V

4

CATGORY

36411COCOA BEAN

210650V

4

CATGORY

36411COCOA BEAN

210650V

4

CATGORY
 



Ask a question (or 'exit'):  number of category 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

076370FOREST

411447DUSTY FIG

073800FOREST

Question: number of category 
Helpful Answer: 6

Question: number of category 

Helpful Answer: 14

Question: number of category 

Helpful Answer: 23

Question: number of category 

Helpful Answer: 31

Question: number of category 

Helpful Answer: 35

Question: number of category 

Helpful Answer: 38

Question: number of category 

Helpful Answer: 40

Question: number of category 

Helpful Answer: 49

Question: number of category 

Helpful Answer: 55

Question: number of category 

Helpful Answer: 58

Question: number of category 

Helpful Answer: 61

Question: number of category 

Helpful Answer: 64

Question: number of category 

Helpful Answer: 65

Question: number of category 

Helpful Answer: 69

Question: 



Ask a question (or 'exit'):  exit


In [ ]:
for i, row in df.iterrows():
    query = row["Category"]  # <-- change the column name if different
    print(f"❓ Question {i+1}: {query}")
    
    # Run the LangChain QA pipeline
    answer = qa.run(query)
    
    print(f"🤖 Answer: {answer}\n")

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.docstore.document import Document
import pandas as pd
df = pd.read_excel(r"C:\Users\User\OneDrive - Aria Intelligent Solutions\MSS White Staff\RLP\Retail Line Print Week 2614.xlsx")
documents = [
    Document(
        page_content=" | ".join(f"{col}: {row[col]}" for col in df.columns),
        metadata={"row": i}
    )
    for i, row in df.iterrows()
]
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
hf_pipeline = pipeline("text-generation", model="gpt2", max_new_tokens=200)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
query = "What are the total sales for menswear?"
result = qa.run(query)
print(result)

In [ ]:
from langchain.text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveChracterTextSplitter(
    chunk_size=100,
    chunk_overlap= 20,
    add_start_index= True,
)

all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)